In [5]:
import random
import numpy as np
import tensorflow as tf
import mlflow.keras

from ml_investing_wne import config
from ml_investing_wne.utils import get_logger
from ml_investing_wne.experiment_factory import create_asset, experiment_factory

random.seed(config.seed)
np.random.seed(config.seed)
tf.random.set_seed(config.seed)

In [6]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

In [7]:
asset = create_asset()
experiment = experiment_factory(asset).get_experiment()

In [9]:
experiment.df.head(2)

,open,high,low,close,volume,y_pred,SMA_5,EMA_5,VAR_5,SMA_10,EMA_10,VAR_10,SMA_15,EMA_15,VAR_15,SMA_20,EMA_20,VAR_20,SMA_50,EMA_50,VAR_50,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,RSI_10,RSI_6,STOCHk_14_3_3,STOCHd_14_3_3,WILLR_14,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0,roc_1,hour,weekday,hour_sin,hour_cos,weekday_sin,weekday_cos
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:49:00,13548.00,13599.99,13545.30,13560.11,6.998959,0.998148,13529.692,13535.457667,1092.81752,13508.876,13520.466610,1067.264782,13508.095333,13514.859063,890.797727,13511.7565,13513.605764,756.107477,13520.3848,13520.384800,3993.977450,1.751876,9.649829,-7.897953,60.980654,66.412127,75.999363,88.106884,76.978855,-26.588439,13470.556426,13529.692,13588.827574,0.874160,0.757189,0.000746,0,0,0.0,1.0,0.0,1.0
2018-01-01 00:50:00,13596.88,13690.87,13521.39,13535.00,16.235999,1.003440,13537.942,13535.305111,691.82627,13512.379,13523.109045,1120.646610,13508.462667,13517.376681,909.662392,13513.5065,13515.643311,774.044056,13516.9264,13520.957945,3268.388191,3.009671,8.726099,-5.716428,54.335462,56.496741,57.714163,66.536466,77.854681,-64.711255,13490.890496,13537.942,13584.993504,0.695106,0.468736,-0.001852,0,0,0.0,1.0,0.0,1.0


In [32]:
df = experiment.df[['open', 'high', 'low', 'close', 'volume']].head(1000)

In [33]:
df['prc_change'] =  df['close']/ df['close'].shift(1) - 1

In [34]:
df

,open,high,low,close,volume,prc_change
datetime,,,,,,
2018-01-01 00:49:00,13548.00,13599.99,13545.30,13560.11,6.998959,NaN
2018-01-01 00:50:00,13596.88,13690.87,13521.39,13535.00,16.235999,-0.001852
2018-01-01 00:51:00,13574.85,13598.99,13542.70,13581.56,2.898871,0.003440
2018-01-01 00:52:00,13598.99,13603.76,13582.48,13598.99,12.245449,0.001283
2018-01-01 00:53:00,13598.99,13598.99,13531.99,13548.20,8.703766,-0.003735
...,...,...,...,...,...,...
2018-01-01 17:24:00,13147.11,13166.00,13136.00,13137.81,1.714353,-0.002763
2018-01-01 17:25:00,13165.98,13165.98,13125.12,13126.63,1.409240,-0.000851
2018-01-01 17:26:00,13126.62,13152.35,13126.62,13149.59,0.445993,0.001749


In [35]:
def cumsum_filter(df,h):
    
    # make index numeric
    df.reset_index(inplace=True)
    group_id,s_pos,s_neg = 0,0,0
    df['prc_change'] =  df['close']/ df['close'].shift(1) - 1
    # get column index of prc_change
    prc_change_index = df.columns.get_loc('prc_change')
    df['group_id'] = 0

    for i in df.index[1:]:
        s_pos,s_neg=max(0,s_pos+df.iloc[i, prc_change_index]),min(0,s_neg+df.iloc[i, prc_change_index])
        df.loc[i, 'group_id'] = group_id
        if s_neg<-h or s_pos>h:
            group_id += 1
            s_pos,s_neg = 0,0
       
    return df



In [36]:
df2 = cumsum_filter(df, 0.01)

In [39]:
df2.to_csv('cumsum_filter.csv')

In [21]:
group_id,s_pos,s_neg = 0,0,0
df['prc_change'] =  df['close']/ df['close'].shift(1) - 1
# get column index of prc_change
prc_change_index = df.columns.get_loc('prc_change')
df['group_id'] = 0


In [29]:
df.reset_index(inplace=True)

In [23]:
i = df.index[1]
print(i)

2018-01-01 00:50:00


In [25]:
prc_change_index

5

In [31]:
df.iloc[1, 4]

13535.0

In [24]:
s_pos,s_neg=max(0,s_pos+df.loc[i, prc_change_index]),min(0,s_neg+df.loc[i, prc_change_index])

KeyError: 5

In [ ]:
for i in df.index[1:]:
    s_pos,s_neg=max(0,s_pos+df.loc[i, prc_change_index]),min(0,s_neg+df.loc[i, prc_change_index])
    df.loc[i, 'group_id'] = group_id
    if s_neg<-h or s_pos>h:
        group_id += 1
        s_pos,s_neg = 0,0